# Master Thesis
## Factor models docs
### v1.1 - description of the YieldCurve package example

We need the data in a format with maturities in columns, and observations in rows. That is, we need to column bind together the maturities we have - 2Y, 5Y, 10Y, 30Y. 

In [9]:
# Example from package YieldCurve
pacman::p_load(YieldCurve)
data(FedYieldCurve) # Fed data set
head(FedYieldCurve) # maturities in columns, observations in rows

Warning message:
"index class is Date, which does not support timezones.
Expected 'UTC' timezone, but indexTZ is ''"

            R_3M  R_6M  R_1Y  R_2Y  R_3Y  R_5Y  R_7Y R_10Y
1981-12-31 12.92 13.90 14.32 14.57 14.64 14.65 14.67 14.59
1982-01-31 14.28 14.81 14.73 14.82 14.73 14.54 14.46 14.43
1982-02-28 13.31 13.83 13.95 14.19 14.13 13.98 13.93 13.86
1982-03-31 13.34 13.87 13.98 14.20 14.18 14.00 13.94 13.87
1982-04-30 12.71 13.13 13.34 13.78 13.77 13.75 13.74 13.62
1982-05-31 13.08 13.76 14.07 14.47 14.48 14.43 14.47 14.30

We may also benefit from the dataset being in the xts format as is the case in the example of YieldCurve package, but if not necessary we'll try to avoid preliminary complexity or optimalisation.

After the dataset is in such a form, we can fit the Nelson-Siegel function and get its parameters.

In [16]:
maturity.Fed <- c(3/12, 0.5, 1,2,3,5,7,10) # define maturities
NSParameters <- Nelson.Siegel(rate=FedYieldCurve,
                               maturity=maturity.Fed)
tail(NSParameters)

Warning message:
"index class is Date, which does not support timezones.
Expected 'UTC' timezone, but indexTZ is ''"

             beta_0    beta_1    beta_2    lambda
2012-06-30 5.942878 -5.805087 -5.918784 0.1839190
2012-07-31 6.007142 -5.890642 -5.770203 0.1938867
2012-08-31 6.877328 -6.739946 -6.945611 0.1839190
2012-09-30 6.752246 -6.624800 -6.609233 0.1839190
2012-10-31 6.292996 -6.173300 -6.111852 0.1839190
2012-11-30 6.590762 -6.496260 -6.364102 0.1839190

Once we have the parameters, we can train the NNs on these parameters as a predictive task. As we can see, we obtain 3 beta coefficients and lambda.

#### termstrc package
The package has been removed from CRAN since 2018.

In [2]:
# Example from package termstrc
install.packages('termstrc')

Installing package into 'C:/Users/Juggle/Documents/R/win-library/3.6'
(as 'lib' is unspecified)
Warning message:
"package 'termstrc' is not available (for R version 3.6.1)"Warning message:
"unable to access index for repository http://www.stats.ox.ac.uk/pub/RWin/bin/windows/contrib/3.6:
  cannot open URL 'http://www.stats.ox.ac.uk/pub/RWin/bin/windows/contrib/3.6/PACKAGES'"

In [4]:
install.packages("termstrc_1.3.7.tar.gz", repos=NULL, type="source")

Installing package into 'C:/Users/Juggle/Documents/R/win-library/3.6'
(as 'lib' is unspecified)
Warning message in install.packages("termstrc_1.3.7.tar.gz", repos = NULL, type = "source"):
"installation of package 'termstrc_1.3.7.tar.gz' had non-zero exit status"

In [6]:
devtools::install_local("termstrc_1.3.7.tar.gz")

Warning message in utils::untar(tarfile, ...):
"'tar.exe -xf "C:\Users\Juggle\AppData\Local\Temp\RtmpohuGNP\file35e0440965b9/termstrc_1.3.7.tar.gz" -C "C:/Users/Juggle/AppData/Local/Temp/RtmpohuGNP/remotes35e0113fc2c"' returned error code 1"

ERROR: Error: Failed to install 'termstrc' from local:
  Does not appear to be an R package (no DESCRIPTION)


We need to build the package from source, from the archive file from CRAN.
 - https://cran.r-project.org/src/contrib/Archive/termstrc/
 - https://stackoverflow.com/questions/51679500/termstrc-was-removed-from-cran-tried-installing-from-tar-file-how-do-i-ins
 - on Win compile w/ Rtools: https://www.biostat.wisc.edu/~kbroman/Rintro/Rwinpack.html

library(termstrc)
data(eurobonds)
Step 2: Set the parameters of the function termstrc as required
countries <- c(" GERMANY ", " AUSTRIA ", " ITALY ")
bonddata <- eurobonds
maturity _ spectrum <- "all"
method <- " Nelson / Siegel "
fit <- " prices "
weights <- " duration "
control <- list ( eval .max =100000)
b <- matrix (c (0.02547394 , -0.012162592 , -0.02547394 , 1 ,
0.02611532 , -0.011367422 , -0.02611532 , 1,
0.02578871 , -0.015207250 , -0.02578871 , 1) ,
nrow =3 , ncol =4 , byrow = TRUE )
rownames (b ) <- countries
colnames (b ) <- c(" beta0 "," beta1 "," beta2 "," tau1 ")
Step 3: Assign the function
myres <- termstrc _ estim ( countries , bonddata , maturity _ spectrum ,
method , fit , weights , startparam =b , control )
Step 4: Use the S3 print method to get the optimised parameter set
print ( myres )
---------------------------------------------------
Parameters for single-curve estimation:
Method: Nelson/Siegel
Fitted: prices
Weights: duration
---------------------------------------------------
GERMANY AUSTRIA ITALY
beta_0 0.04153832 0.04228676 0.04599300
beta_1 -0.01699020 -0.01813795 -0.02089901
beta_2 -0.01326502 -0.01208279 -0.01874076
tau_1 2.20819586 2.48874374 2.36039421
Step 5: Use the S3 summary method to get the goodness of fit test and the convergence information
summary ( myres )
12 termstrc: A Package for Term Structure Estimation with R
---------------------------------------------------
Goodness of fit tests:
---------------------------------------------------
GERMANY AUSTRIA ITALY
RMSE-Prices 0.2852092109 0.0606758664 0.1588754369
AABSE-Prices 0.1629764768 0.0416447015 0.1041235559
RMSE-Yields 0.0004808922 0.0003668975 0.0007122635
AABSE-Yields 0.0004177424 0.0002694297 0.0006036436
Convergence info
GERMANY "relative convergence (4)"
AUSTRIA "relative convergence (4)"
ITALY "relative convergence (4)"
Step 6: Use the S3 plot method to generate different plots of the termstructure and the spreadcurves. By default the zero-coupon-yield curve is plotted separately for each specified country and
for all countries together. Morover the spread-curves are plotted.
plot ( myres )
Figure 3: Zero-coupon yield curve

## Sorted:
Available packages in R to take inspiration from:
 - [YieldCurve](https://cran.r-project.org/web/packages/YieldCurve/YieldCurve.pdf)
    - [Nelson.Siegel](https://www.rdocumentation.org/packages/YieldCurve/versions/4.1/topics/Nelson.Siegel) function, its code [here](https://rdrr.io/rforge/YieldCurve/src/R/Nelson.Siegel.R)
 - [termstrc](https://r-forge.r-project.org/scm/viewvc.php/*checkout*/termstrc-vignette/termstrc.pdf?revision=2&root=termstrc&pathrev=2)
    - [RDocumentation](https://www.rdocumentation.org/packages/termstrc/versions/1.3.7)
    - package removed from CRAN

## Sort out:

## Factor models in R

* [DEnss.pdf](https://cran.r-project.org/web/packages/NMOF/vignettes/DEnss.pdf)
* [download.cgi](https://editorialexpress.com/cgi-bin/conference/download.cgi?db_name=XVIEBFIN&paper_id=33)
* [Forecasting the yield curve with the arbitrage-free dynamic Nelson–Siegel model: Brazilian evidence - ScienceDirect](https://www.sciencedirect.com/science/article/pii/S1517758016300789)
* [Calibrating the Dynamic Nelson-Siegel Model: A Practitioner Approach - Munich Personal RePEc Archive](https://mpra.ub.uni-muenchen.de/68439/)
* [ijfs-06-00068.pdf](file:///E:/Google_Drive/Diploma_Thesis/Sources/Random_Google/Factor%20models/ijfs-06-00068.pdf)
* [The Nelson-Siegel Model: Present Application and Alternative Lambda Determination](file:///E:/Google_Drive/Diploma_Thesis/Sources/Random_Google/Factor%20models/BPTX_2013_2_11230_0_387924_0_151485.pdf)
